In [7]:
import requests
import pandas as pd
import pickle
from statistics import mean
import os
import json
import yfinance as yf
import matplotlib.pyplot as plt

In [8]:
def apply_portafolio_theory(tickers_composition,dollar_price,interval,period):
    tickers_statistics={}
    total_invested=sum(x[1]*x[2] for x in  tickers_composition)            
    earnings_df = pd.DataFrame()
    portfolio_earning=0
    for ticker,quantity,current_price in tickers_composition:
        #Getting earnings usgin historicals per ticker 
        hist = yf.Ticker(ticker).history(period=period,interval=interval).dropna()
        hist["earnings"]=(hist.Close-hist.Open)*100/hist.Open
        tickers_statistics[ticker]={ \
                                    "earning":hist.earnings.mean(),\
                                    "risk":hist.earnings.std(),\
                                    "performance":hist.earnings.mean()/hist.earnings.std(),\
                                    "portfolio_percentage":current_price*quantity/total_invested \
                                   }
        earnings_df=pd.concat([earnings_df,hist["earnings"]],axis = 1)
        earnings_df.rename(columns = {'earnings':ticker}, inplace = True)
        #Getting portfolio earning using earnings mean and portfolio percentage per ticker
        portfolio_earning+=tickers_statistics[ticker]['earning']*tickers_statistics[ticker]['portfolio_percentage']

    portfolio_risk=0
    cov_matrix=(earnings_df).cov().values
    
    for x in range(len(earnings_df.cov().values)):
        for y in range(len(earnings_df.cov().values[x])):
            portfolio_risk+=(tickers_composition[x][1]*tickers_composition[x][2]/total_invested)*(tickers_composition[y][1]*tickers_composition[y][2]/total_invested)*cov_matrix[x][y]
    portfolio_risk=portfolio_risk**(1/2)
    portfolio_performance=portfolio_earning/portfolio_risk

#    print(f"Portfolio earnings per {interval}: {portfolio_earning} \
#            \nPortfolio risk per {interval}: {portfolio_risk} \
#            \nPortfolio performance: {portfolio_performance} \
#          ")
    return portfolio_performance,portfolio_earning,portfolio_risk


In [9]:
##S&P500 List
a='''
MMM
AOS
ABT
ABBV
ACN
ATVI
ADM
ADBE
ADP
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ANET
AJG
AIZ
T
ATO
ADSK
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BBWI
BAX
BDX
WRB
BRK.B 
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B 
BG
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISH
DIS
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
RE
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FITB
FSLR
FE
FIS
FISV
FLT
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GEHC
GEN
GNRC
GD
GE
GIS
GM
GPC
GILD
GL
GPN
GS
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUM
HBAN
HII
IBM
IEX
IDXX
ITW
ILMN
INCY
IR
PODD
INTC
ICE
IFF
IP
IPG
INTU
ISRG
IVZ
INVH
IQV
IRM
JBHT
JKHY
J
JNJ
JCI
JPM
JNPR
K
KDP
KEY
KEYS
KMB
KIM
KMI
KLAC
KHC
KR
LHX
LH
LRCX
LW
LVS
LDOS
LEN
LNC
LIN
LYV
LKQ
LMT
L
LOW
LYB
MTB
MRO
MPC
MKTX
MAR
MMC
MLM
MAS
MA
MTCH
MKC
MCD
MCK
MDT
MRK
META
MET
MTD
MGM
MCHP
MU
MSFT
MAA
MRNA
MHK
MOH
TAP
MDLZ
MPWR
MNST
MCO
MS
MOS
MSI
MSCI
NDAQ
NTAP
NFLX
NWL
NEM
NWSA
NWS
NEE
NKE
NI
NDSN
NSC
NTRS
NOC
NCLH
NRG
NUE
NVDA
NVR
NXPI
ORLY
OXY
ODFL
OMC
ON
OKE
ORCL
OGN
OTIS
PCAR
PKG
PARA
PH
PAYX
PAYC
PYPL
PNR
PEP
PKI
PFE
PCG
PM
PSX
PNW
PXD
PNC
POOL
PPG
PPL
PFG
PG
PGR
PLD
PRU
PEG
PTC
PSA
PHM
QRVO
PWR
QCOM
DGX
RL
RJF
RTX
O
REG
REGN
RF
RSG
RMD
RHI
ROK
ROL
ROP
ROST
RCL
SPGI
CRM
SBAC
SLB
STX
SEE
SRE
NOW
SHW
SPG
SWKS
SJM
SNA
SEDG
SO
LUV
SWK
SBUX
STT
STLD
STE
SYK
SYF
SNPS
SYY
TMUS
TROW
TTWO
TPR
TRGP
TGT
TEL
TDY
TFX
TER
TSLA
TXN
TXT
TMO
TJX
TSCO
TT
TDG
TRV
TRMB
TFC
TYL
TSN
USB
UDR
ULTA
UNP
UAL
UPS
URI
UNH
UHS
VLO
VTR
VRSN
VRSK
VZ
VRTX
VFC
VTRS
VICI
V
VMC
WAB
WBA
WMT
WBD
WM
WAT
WEC
WFC
WELL
WST
WDC
WRK
WY
WHR
WMB
WTW
GWW
WYNN
XEL
XYL
YUM
ZBRA
ZBH
ZION
ZTS
'''

In [10]:
results=[]
try_tickers=a.split()
dollar_price=18.42
interval="1wk"
period="5y"
money_to_invest=10000
tickers_composition=[("AAPL",1),
                     ("AMZN",7) ,\
                     ("CDNS",1) ,\
                     ("GOOGL",20), \
                     ("LLY",1), \
                     ("META",1), \
                     ("MSFT",1), \
                     ("NEE",1), \
                     ("PG",1), \
                     ("TMUS",1), \
                     ("TSLA",1), \
                     ("V",1), \
                     ("VOO",1), \
                     ("XLK",1), \
                     
                     
]
ticker_and_price=[ (ticker[0],ticker[1], yf.Ticker(ticker[0]).history(period="1d",interval="1d").iloc[0]['Close']*dollar_price) for ticker in tickers_composition ]

for ticker in try_tickers:
    try:
        print("Using: "+ticker)
        ticker_candidate_price=yf.Ticker(ticker).history(period="1d",interval="1d").iloc[0]['Close']*dollar_price
        if ticker_candidate_price > money_to_invest:
            print(ticker+" is to expensive. Moving to next one")
            continue
        else:
            ticker_and_price.append((ticker, 1,ticker_candidate_price))
            p_perf,p_earn,p_risk=apply_portafolio_theory(ticker_and_price,dollar_price,interval,period)
            results.append((ticker,p_perf,p_earn,p_risk)) 
            ticker_and_price.pop()
    except :
        print ("Failed with: "+ticker)
        ticker_and_price.pop()
print("-----FINISHED--------")

Using: MMM
Using: AOS
Using: ABT
Using: ABBV
Using: ACN
Using: ATVI


ATVI: No data found, symbol may be delisted


Failed with: ATVI
Using: ADM
Using: ADBE
Using: ADP
Using: AAP
Using: AES
Using: AFL
Using: A
Using: APD
Using: AKAM
Using: ALK
Using: ALB
Using: ARE
Using: ALGN
Using: ALLE
Using: LNT
Using: ALL
Using: GOOGL
Using: GOOG
Using: MO
Using: AMZN
Using: AMCR
Using: AMD
Using: AEE
Using: AAL
Using: AEP
Using: AXP
Using: AIG
Using: AMT
Using: AWK
Using: AMP
Using: ABC


ABC: No data found, symbol may be delisted


Failed with: ABC
Using: AME
Using: AMGN
Using: APH
Using: ADI
Using: ANSS
Using: AON
Using: APA
Using: AAPL
Using: AMAT
Using: APTV
Using: ACGL
Using: ANET
Using: AJG
Using: AIZ
Using: T
Using: ATO
Using: ADSK
Using: AZO
AZO is to expensive. Moving to next one
Using: AVB
Using: AVY
Using: AXON
Using: BKR
Using: BALL
Using: BAC
Using: BBWI
Using: BAX
Using: BDX
Using: WRB
Using: BRK.B


...

Failed with: PEAK
Using: HSIC
Using: HSY
Using: HES
Using: HPE
Using: HLT
Using: HOLX
Using: HD
Using: HON
Using: HRL
Using: HST
Using: HWM
Using: HPQ
Using: HUM
Using: HBAN
Using: HII
Using: IBM
Using: IEX
Using: IDXX
Using: ITW
Using: ILMN
Using: INCY
Using: IR
Using: PODD
Using: INTC
Using: ICE
Using: IFF
Using: IP
Using: IPG
Using: INTU
INTU is to expensive. Moving to next one
Using: ISRG
Using: IVZ
Using: INVH
Using: IQV
Using: IRM
Using: JBHT
Using: JKHY
Using: J
Using: JNJ
Using: JCI
Using: JPM
Using: JNPR
Using: K
Using: KDP
Using: KEY
Using: KEYS
Using: KMB
Using: KIM
Using: KMI
Using: KLAC
KLAC is to expensive. Moving to next one
Using: KHC
Using: KR
Using: LHX
Using: LH
Using: LRCX
LRCX is to expensive. Moving to next one
Using: LW
Using: LVS
Using: LDOS
Using: LEN
Using: LNC
Using: LIN
Using: LYV
Using: LKQ
Using: LMT
Using: L
Using: LOW
Using: LYB
Using: MTB
Using: MRO
Using: MPC
Using: MKTX
Using: MAR
Using: MMC
Using: MLM
MLM is to expensive. Moving to next one
Using: MA

PKI: No data found, symbol may be delisted


Failed with: PKI
Using: PFE
Using: PCG
Using: PM
Using: PSX
Using: PNW
Using: PXD


$PXD: possibly delisted; No price data found  (period=1d)


$PXD: possibly delisted; No price data found  (period=1d)
Failed with: PXD
Using: PNC
Using: POOL
Using: PPG
Using: PPL
Using: PFG
Using: PG
Using: PGR
Using: PLD
Using: PRU
Using: PEG
Using: PTC
Using: PSA
Using: PHM
Using: QRVO
Using: PWR
Using: QCOM
Using: DGX
Using: RL
Using: RJF
Using: RTX
Using: O
Using: REG
Using: REGN
REGN is to expensive. Moving to next one
Using: RF
Using: RSG
Using: RMD
Using: RHI
Using: ROK
Using: ROL
Using: ROP
ROP is to expensive. Moving to next one
Using: ROST
Using: RCL
Using: SPGI
Using: CRM
Using: SBAC
Using: SLB
Using: STX
Using: SEE
Using: SRE
Using: NOW
NOW is to expensive. Moving to next one
Using: SHW
Using: SPG
Using: SWKS
Using: SJM
Using: SNA
Using: SEDG
Using: SO
Using: LUV
Using: SWK
Using: SBUX
Using: STT
Using: STLD
Using: STE
Using: SYK
Using: SYF
Using: SNPS
SNPS is to expensive. Moving to next one
Using: SYY
Using: TMUS
Using: TROW
Using: TTWO
Using: TPR
Using: TRGP
Using: TGT
Using: TEL
Using: TDY
Using: TFX
Using: TER
Using: TSLA
Usin

In [11]:
pd.DataFrame(results).sort_values(by=[1],ascending=False).head(20)

,0,1,2,3
117,CEG,0.227578,0.677107,2.975279
46,ANET,0.222020,0.657282,2.960460
86,CAT,0.221295,0.641665,2.899586
47,AJG,0.221198,0.644773,2.914910
152,ETN,0.220451,0.663033,3.007620
43,AMAT,0.220366,0.656460,2.978950
42,AAPL,0.220203,0.651249,2.957490
296,MRNA,0.219962,0.684608,3.112399
323,NVDA,0.219923,0.693865,3.155032
36,AMGN,0.219861,0.634218,2.884634


In [ ]:
look_details=('CHKEW', 1, 'NA', 10000)
tickers_composition.append(look_details)
print(apply_portafolio_theory(tickers_composition))
tickers_composition.pop()

(0.15855428460132498, 0.4671408000647136, 2.94625150773636)


('CHKEW', 1, 'NA', 10000)

In [ ]:
tickers_composition

[('AAPL', 1, 2470, 2733.63),
 ('AMZN', 7, 2000, 1670.18),
 ('GOOGL', 20, 2760, 1668.34),
 ('LLY', 1, 3400, 5798.98),
 ('META', 1, 6600, 3304.47),
 ('MSFT', 1, 6000, 4576.11),
 ('NEE', 1, 1500, 1343.98),
 ('PG', 1, 2950, 2525.43),
 ('TMUS', 1, 2567.68, 2568.14),
 ('V', 1, 4420, 3978.76),
 ('VOO', 1, 7571, 6529.05),
 ('XLK', 1, 2282, 2499.29)]